In [2]:
import h5py
import pandas
import numpy
import os
import glob
from collections import defaultdict
import trodes.read_exported
import pandas as pd
import numpy as np
from scipy import stats
from spectral_connectivity import Multitaper, Connectivity
import logging
import h5py
import pickle
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):   
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl 
    Returns:
        none
    """
    with open(file_name,'wb') as file:
        pickle.dump(thing_to_pickle, file)

def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):   
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, 'rb') as file:
        return(pickle.load(file))

In [3]:
tester_df = unpickle_this(r"D:\social_ephys_pilot2_cum\data\phase2\20230818_133620_1.4_CNF.rec\outputs\lfp_all_df.pkl")
tester_df

,accelerometer,autosettle,clock rate,clockrate,cohort,system_time_at_creation,timestamp_at_creation,data,first_item_data,last_item_data,...,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,LH_MD_granger_all_frequencies_all_windows,MD_LH_granger_all_frequencies_all_windows,LH_vHPC_granger_all_frequencies_all_windows,vHPC_LH_granger_all_frequencies_all_windows,MD_vHPC_granger_all_frequencies_all_windows,vHPC_MD_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies
0,0,0,NaN,20000,2,1692380198431,1024156,"[[1923587], [1923588], [1923589], [1923590], [...","[1923587, 1923588, 1923589, 1923590, 1923591, ...","[1923587, 1923588, 1923589, 1923590, 1923591, ...",...,"[[0.1443, 0.0874, 0.02318, 0.048, 0.01758, 0.1...","[[0.36, 0.3413, 0.2373, 0.1075, 0.04514, 0.518...","[[0.944, 0.292, 0.00887, 0.001174, 0.1242, 0.3...","[[0.3025, 0.3945, 0.0641, 0.1993, 0.371, 0.385...","[[0.355, 0.2225, 0.2854, 0.226, 0.308, 1.057, ...","[[1.241, 1.95, 1.188, 0.8027, 0.802, 1.043, 0....","[[0.4856, 0.2389, 0.312, 0.2212, 0.3452, 0.758...","[[2.023, 1.648, 1.169, 0.854, 0.8403, 1.096, 1...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


looks like data, first_item_data, and last_item_data are the same things...
sample numbers x3 

In [77]:
tester_df.columns[0:38]

Index(['accelerometer', 'autosettle', 'clock rate', 'clockrate', 'cohort',
       'system_time_at_creation', 'timestamp_at_creation', 'data',
       'first_item_data', 'last_item_data', 'compile_date', 'description',
       'session_dir', 'metadata_dir', 'fields', 'metadata_file',
       'original_file', 'filename', 'controller_firmware',
       'headstage_firmware', 'gyro', 'magnetometer', 'first_dtype_name',
       'last_dtype_name', 'time_offset', 'byte_order', 'session_path',
       'recording', 'controller_serialnum', 'headstage_serialnum', 'smartref',
       'current_subject', 'subject', 'all_subjects', 'commit_tag',
       'compile_time', 'first_timestamp', 'lfp_timestamps'],
      dtype='object')

In [109]:
tester_df.loc[:, 'first_timestamp']

0    1923587
Name: first_timestamp, dtype: int64

first_item_data should be dependent on the length of the items in data 


In [124]:
metadata_trodes = tester_df.iloc[:, 0:38]
metadata_trodes = metadata_trodes[['data',
                                   'session_dir',
                                   'original_file',
                                   'session_path',
                                   'subject', 
                                   'first_timestamp']]

metadata_trodes['data'] = metadata_trodes.at[0,'data'].shape[0]
metadata_trodes

,data,session_dir,original_file,session_path,subject,first_timestamp
0,42011249,20230818_133620_1.4_CNF,20230818_133620_1_merged.rec,D:\social_ephys_pilot2_cum\data\tester\2023081...,1.4,1923587


In [125]:
columns_to_stack = []
for col in tester_df.columns[38:43]:
    for index, value in tester_df[col].items():
        print(f"Row {index}, Column {col}: ", end="")
        columns_to_stack.append(col)
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

Row 0, Column mPFC_lfp_trace: numpy array with shape (2100535,)
Row 0, Column MD_lfp_trace: numpy array with shape (2100535,)
Row 0, Column LH_lfp_trace: numpy array with shape (2100535,)
Row 0, Column BLA_lfp_trace: numpy array with shape (2100535,)
Row 0, Column vHPC_lfp_trace: numpy array with shape (2100535,)


In [136]:
raw_lfp_array = np.vstack([tester_df[col].values[0] for col in columns_to_stack])
raw_lfp_array.shape

(5, 2100535)

raw lfp traces

In [75]:
tester_df.iloc[:,43:45]

,trodes_version,qt_version
0,2.3.4,6.2.2


In [74]:
tester_df.iloc[:, 45:50]

,mPFC_lfp_MAD,MD_lfp_MAD,LH_lfp_MAD,BLA_lfp_MAD,vHPC_lfp_MAD
0,54.404999,42.119999,40.169998,46.41,58.695


median absolute deviation used for normalizing the signals

In [140]:
columns_to_stack = []
for col in tester_df.columns[50:55]:
    for index, value in tester_df[col].items():
        columns_to_stack.append(col)
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

zscore_lfp_array = np.vstack([tester_df[col].values[0] for col in columns_to_stack])
zscore_lfp_array.shape

Row 0, Column mPFC_lfp_modified_zscore: numpy array with shape (2100535,)
Row 0, Column MD_lfp_modified_zscore: numpy array with shape (2100535,)
Row 0, Column LH_lfp_modified_zscore: numpy array with shape (2100535,)
Row 0, Column BLA_lfp_modified_zscore: numpy array with shape (2100535,)
Row 0, Column vHPC_lfp_modified_zscore: numpy array with shape (2100535,)


(5, 2100535)

these are normalized traces using z score to remove noise and then used as input to RMS


In [30]:
tester_df.columns[55:60]

Index(['mPFC_lfp_RMS', 'MD_lfp_RMS', 'LH_lfp_RMS', 'BLA_lfp_RMS',
       'vHPC_lfp_RMS'],
      dtype='object')

In [139]:
columns_to_stack = []
for col in tester_df.columns[55:60]:
    for index, value in tester_df[col].items():
        columns_to_stack.append(col)
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")
rms_lfp_array = np.vstack([tester_df[col].values[0] for col in columns_to_stack])
rms_lfp_array.shape            

Row 0, Column mPFC_lfp_RMS: numpy array with shape (2100535,)
Row 0, Column MD_lfp_RMS: numpy array with shape (2100535,)
Row 0, Column LH_lfp_RMS: numpy array with shape (2100535,)
Row 0, Column BLA_lfp_RMS: numpy array with shape (2100535,)
Row 0, Column vHPC_lfp_RMS: numpy array with shape (2100535,)


(5, 2100535)

this allows us to compare across animals and this is what we calulcate power on 
root square of the mean

In [95]:
tester_df.iloc[:,60:65]

,mPFC_lfp_mask,MD_lfp_mask,LH_lfp_mask,BLA_lfp_mask,vHPC_lfp_mask
0,"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal..."


In [141]:
columns_to_stack = []
for col in tester_df.columns[60:65]:
    for index, value in tester_df[col].items():
        columns_to_stack.append(col)
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")


Row 0, Column mPFC_lfp_mask: numpy array with shape (2100535,)
Row 0, Column MD_lfp_mask: numpy array with shape (2100535,)
Row 0, Column LH_lfp_mask: numpy array with shape (2100535,)
Row 0, Column BLA_lfp_mask: numpy array with shape (2100535,)
Row 0, Column vHPC_lfp_mask: numpy array with shape (2100535,)


(5, 2100535)

these are boolean numpy arrays false = no noise, true = noise

In [80]:
for col in tester_df.columns[65:70]:
    for index, value in tester_df[col].items():
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

Row 0, Column mPFC_lfp_trace_filtered: numpy array with shape (2100535,)
Row 0, Column MD_lfp_trace_filtered: numpy array with shape (2100535,)
Row 0, Column LH_lfp_trace_filtered: numpy array with shape (2100535,)
Row 0, Column BLA_lfp_trace_filtered: numpy array with shape (2100535,)
Row 0, Column vHPC_lfp_trace_filtered: numpy array with shape (2100535,)


this is filtered for noise but not RMS-ed

In [142]:
columns_to_stack = []
for col in tester_df.columns[70:75]:
    for index, value in tester_df[col].items():
        columns_to_stack.append(col)
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")
rms_filtered_lfp_array = np.vstack([tester_df[col].values[0] for col in columns_to_stack])
rms_filtered_lfp_array.shape 

Row 0, Column mPFC_lfp_RMS_filtered: numpy array with shape (2100535,)
Row 0, Column MD_lfp_RMS_filtered: numpy array with shape (2100535,)
Row 0, Column LH_lfp_RMS_filtered: numpy array with shape (2100535,)
Row 0, Column BLA_lfp_RMS_filtered: numpy array with shape (2100535,)
Row 0, Column vHPC_lfp_RMS_filtered: numpy array with shape (2100535,)


(5, 2100535)

filtered for noise with RMS 

In [82]:
for col in tester_df.columns[75:95]:
    for index, value in tester_df[col].items():
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

Row 0, Column mPFC_trace_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column MD_trace_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column LH_trace_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column BLA_trace_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column vHPC_trace_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column mPFC_RMS_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column MD_RMS_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column LH_RMS_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column BLA_RMS_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column vHPC_RMS_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column mPFC_trace_filtered_power_all_frequencies_all_windows: n

the only columns i should be looking at are RMS_Filtered_power 
2 hz time windows 

42011249 is taken 20000 Hz
2100535 is 1000 Hz (LFP down sample)

1 sec with .5 overlap --> we have 2 data points per second 
500 frequencies (which are listed below) 

In [154]:
columns_to_stack = []
for col in tester_df.columns[90:95]:
    for index, value in tester_df[col].items():
        columns_to_stack.append(col)
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

power_lfp = np.stack([tester_df[col].values[0] for col in columns_to_stack], axis = 0)
power_lfp.shape

Row 0, Column mPFC_RMS_filtered_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column MD_RMS_filtered_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column LH_RMS_filtered_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column BLA_RMS_filtered_power_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column vHPC_RMS_filtered_power_all_frequencies_all_windows: numpy array with shape (4200, 500)


(5, 4200, 500)

In [84]:
for col in tester_df.columns[95:97]:
    for index, value in tester_df[col].items():
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

Row 0, Column power_timestamps: numpy array with shape (4200,)
Row 0, Column power_calculation_frequencies: numpy array with shape (500,)


In [85]:
for col in tester_df.columns[97:107]:
    for index, value in tester_df[col].items():
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

Row 0, Column mPFC_theta_band: numpy array with shape (2100535,)
Row 0, Column MD_theta_band: numpy array with shape (2100535,)
Row 0, Column LH_theta_band: numpy array with shape (2100535,)
Row 0, Column BLA_theta_band: numpy array with shape (2100535,)
Row 0, Column vHPC_theta_band: numpy array with shape (2100535,)
Row 0, Column mPFC_gamma_band: numpy array with shape (2100535,)
Row 0, Column MD_gamma_band: numpy array with shape (2100535,)
Row 0, Column LH_gamma_band: numpy array with shape (2100535,)
Row 0, Column BLA_gamma_band: numpy array with shape (2100535,)
Row 0, Column vHPC_gamma_band: numpy array with shape (2100535,)


should add documentation for what band theta is 

In [49]:
tester_df.columns[107:117]

Index(['mPFC_theta_phase', 'MD_theta_phase', 'LH_theta_phase',
       'BLA_theta_phase', 'vHPC_theta_phase', 'mPFC_gamma_phase',
       'MD_gamma_phase', 'LH_gamma_phase', 'BLA_gamma_phase',
       'vHPC_gamma_phase'],
      dtype='object')

In [86]:
for col in tester_df.columns[107:117]:
    for index, value in tester_df[col].items():
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

Row 0, Column mPFC_theta_phase: numpy array with shape (2100535,)
Row 0, Column MD_theta_phase: numpy array with shape (2100535,)
Row 0, Column LH_theta_phase: numpy array with shape (2100535,)
Row 0, Column BLA_theta_phase: numpy array with shape (2100535,)
Row 0, Column vHPC_theta_phase: numpy array with shape (2100535,)
Row 0, Column mPFC_gamma_phase: numpy array with shape (2100535,)
Row 0, Column MD_gamma_phase: numpy array with shape (2100535,)
Row 0, Column LH_gamma_phase: numpy array with shape (2100535,)
Row 0, Column BLA_gamma_phase: numpy array with shape (2100535,)
Row 0, Column vHPC_gamma_phase: numpy array with shape (2100535,)


In [63]:
tester_df.columns[117:125]

Index(['mPFC_LH_trace_filtered_coherence_all_frequencies_all_windows',
       'mPFC_LH_trace_coherence_all_frequencies_all_windows',
       'mPFC_LH_RMS_filtered_coherence_all_frequencies_all_windows',
       'mPFC_LH_RMS_coherence_all_frequencies_all_windows',
       'mPFC_LH_trace_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows',
       'mPFC_LH_RMS_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows',
       'mPFC_LH_RMS_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows',
       'mPFC_LH_trace_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows'],
      dtype='object')

In [87]:
for col in tester_df.columns[117:125]:
    for index, value in tester_df[col].items():
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

Row 0, Column mPFC_LH_trace_filtered_coherence_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column mPFC_LH_trace_coherence_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column mPFC_LH_RMS_filtered_coherence_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column mPFC_LH_RMS_coherence_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column mPFC_LH_trace_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows: numpy array with shape (0, 500)
Row 0, Column mPFC_LH_RMS_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows: numpy array with shape (0, 500)
Row 0, Column mPFC_LH_RMS_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows: numpy array with shape (0, 500)
Row 0, Column mPFC_LH_trace_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows: numpy array with shape (0, 500)


in theory - this should only be done on the RMS filtered although in theory in theory i should check if the filter is good for my data 
a parameter should be the number of SD to be filtered out , with a default of 4, another function to be added to this class should be plotting given brain regions of a zscored data filtered 
also there should be only column for the calculation made for frequencies across all your brain regions i.e. power frequencies, coherence frequencies etc. 

In [106]:
tester_df.iloc[:,117]

0    [[0.866, 0.7554, 0.7554, 0.6367, 0.68, 0.882, ...
Name: mPFC_LH_trace_filtered_coherence_all_frequencies_all_windows, dtype: object

In [103]:
val = tester_df['mPFC_LH_trace_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows']
print(val)

0    []
Name: mPFC_LH_trace_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows, dtype: object


In [65]:
tester_df.columns[125:133]

Index(['mPFC_BLA_trace_filtered_coherence_all_frequencies_all_windows',
       'mPFC_BLA_trace_coherence_all_frequencies_all_windows',
       'mPFC_BLA_RMS_filtered_coherence_all_frequencies_all_windows',
       'mPFC_BLA_RMS_coherence_all_frequencies_all_windows',
       'mPFC_BLA_trace_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows',
       'mPFC_BLA_RMS_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows',
       'mPFC_BLA_RMS_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows',
       'mPFC_BLA_trace_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows'],
      dtype='object')

In [67]:
tester_df.columns[133:141]

Index(['mPFC_MD_trace_filtered_coherence_all_frequencies_all_windows',
       'mPFC_MD_trace_coherence_all_frequencies_all_windows',
       'mPFC_MD_RMS_filtered_coherence_all_frequencies_all_windows',
       'mPFC_MD_RMS_coherence_all_frequencies_all_windows',
       'mPFC_MD_trace_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows',
       'mPFC_MD_RMS_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows',
       'mPFC_MD_RMS_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows',
       'mPFC_MD_trace_filtered_power_all_frequencies_all_windows_coherence_all_frequencies_all_windows'],
      dtype='object')

In [92]:
for col in tester_df.columns[197:199]:
    for index, value in tester_df[col].items():
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

Row 0, Column coherence_timestamps: numpy array with shape (4200,)
Row 0, Column coherence_calculation_frequencies: numpy array with shape (500,)


In [93]:
for col in tester_df.columns[199:221]:
    for index, value in tester_df[col].items():
        print(f"Row {index}, Column {col}: ", end="")
        if isinstance(value, np.ndarray):
            print(f"numpy array with shape {value.shape}")
        else:
            print(f"type {type(value)}")

Row 0, Column BLA_mPFC_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column mPFC_BLA_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column BLA_LH_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column LH_BLA_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column BLA_MD_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column MD_BLA_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column BLA_vHPC_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column vHPC_BLA_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column mPFC_LH_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column LH_mPFC_granger_all_frequencies_all_windows: numpy array with shape (4200, 500)
Row 0, Column mPFC_MD_granger_all_frequencies_all_windows: n